In [1]:
import os
os.environ["ZHIPUAI_API_KEY"] = '056744e429f9da058272391d1107ebe6.7aXbE4ZlIuYn7yoL'

import re
import asyncio
from metagpt.actions import Action
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.logs import logger

2024-05-17 02:39:58.423 | INFO     | metagpt.const:get_metagpt_package_root:32 - Package root set to /root/autodl-tmp/MetaGPT
2024-05-17 02:39:58.756 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI


In [4]:
class SimpleWriteCode(Action):

    PROMPT_TEMPLATE: str = """
    Write a python function that can {instruction} and provide two runnable test cases.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """

    name: str = "SimpleWriteCode"

    async def run(self, instruction: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)
        rsp = await self._aask(prompt)
        code_text = SimpleWriteCode.parse_code(rsp)
        return code_text

    @staticmethod
    def parse_code(rsp):
        pattern = r'```python(.*)```'
        match = re.search(pattern, rsp, re.DOTALL)
        code_text = match.group(1) if match else rsp
        return code_text

class SimpleCoder(Role):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._init_actions([SimpleWriteCode])

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: ready to {self.rc.todo}")
        todo = self.rc.todo  # todo will be SimpleWriteCode()

        msg = self.get_memories(k=1)[0]  # find the most recent messages
        code_text = await todo.run(msg.content)
        msg = Message(content=code_text, role=self.profile,
                      cause_by=type(todo))

        return msg

async def main():
    msg = "write a function that calculates the sum of a list"
    role = SimpleCoder()
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)

await main()

2024-05-17 02:52:58.173 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI
2024-05-17 02:52:58.176 | INFO     | __main__:main:45 - write a function that calculates the sum of a list
2024-05-17 02:52:58.179 | INFO     | __main__:_act:30 - (): ready to SimpleWriteCode


[user: write a function that calculates the sum of a list]
user: write a function that calculates the sum of a list
 ```python
def sum_of_list(lst):
    return sum(lst)

# Test Case 1
test_list1 = [1, 2, 3, 4, 5]
expected_result1 = 15
assert sum_of_list(test_list1) == expected_result1, f"Expected {expected_result1} but got {sum_of_list(test_list1)}"

# Test Case 2
test_list2 = [10, -5, 20, 0, 5]
expected_result2 = 20
assert sum_of_list(test_list2) == expected_result2, f"Expected {expected_result2} but got {sum_of_list(test_list2)}"

print("Both test cases passed.")
```

Your code:

```python
def sum_of_list(lst):
    return sum(lst)

# Test Case 1
test_list1 = [1, 2, 3, 4, 5]
expected_result1 = 15
assert sum_of_list(test_list1) == expected_result1, f"Expected {expected_result1} but got {sum_of_list(test_list1)}"

# Test Case 2
test_list2 = [10, -5, 20, 0, 5]
expected_result2 = 20
assert sum_of_list(test_list2) == expected_result2, f"Expected {expected_result2} but got {sum_of_list(test

2024-05-17 02:53:19.289 | INFO     | metagpt.utils.cost_manager:update_cost:48 - Total running cost: $0.001 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 52, completion_tokens: 404
2024-05-17 02:53:19.291 | INFO     | __main__:main:47 - : 
def sum_of_list(lst):
    return sum(lst)

# Test Case 1
test_list1 = [1, 2, 3, 4, 5]
expected_result1 = 15
assert sum_of_list(test_list1) == expected_result1, f"Expected {expected_result1} but got {sum_of_list(test_list1)}"

# Test Case 2
test_list2 = [10, -5, 20, 0, 5]
expected_result2 = 20
assert sum_of_list(test_list2) == expected_result2, f"Expected {expected_result2} but got {sum_of_list(test_list2)}"

print("Both test cases passed.")
```

Your code:

```python
def sum_of_list(lst):
    return sum(lst)

# Test Case 1
test_list1 = [1, 2, 3, 4, 5]
expected_result1 = 15
assert sum_of_list(test_list1) == expected_result1, f"Expected {expected_result1} but got {sum_of_list(test_list1)}"

# Test Case 2
test_list2 = [10, -5, 20, 0, 5]
e

`

In [5]:
class SimpleRunCode(Action):

    name: str = "SimpleRunCode"

    async def run(self, code_text: str):
        # 在Windows环境下，result可能无法正确返回生成结果，在windows中在终端中输入python3可能会导致打开微软商店
        result = subprocess.run(["python3", "-c", code_text], capture_output=True, text=True)
        # 采用下面的可选代码来替换上面的代码
        # result = subprocess.run(["python", "-c", code_text], capture_output=True, text=True)
        # import sys
        # result = subprocess.run([sys.executable, "-c", code_text], capture_output=True, text=True)
        code_result = result.stdout
        logger.info(f"{code_result=}")
        return code_result

    name: str = "SimpleRunCode"
    async def run(self, code_text: str):
        result = subprocess.run(["python3", "-c", code_text], capture_output = True, text = True)
        code_result = result.stdout
        logger.info(f"{code_result=}")
        return code_result

In [7]:
import subprocess
import fire
# from metagpt.llm import LLM

class RunnableCoder(Role):

    name: str = "Alice"
    profile: str = "RunnableCoder"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._init_actions([SimpleWriteCode, SimpleRunCode])
        self._set_react_mode(react_mode="by_order")

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: 准备 {self.rc.todo}")
        # 通过在底层按顺序选择动作
        # todo 首先是 SimpleWriteCode() 然后是 SimpleRunCode()
        todo = self.rc.todo

        msg = self.get_memories(k=1)[0] # 得到最相似的 k 条消息
        result = await todo.run(msg.content)

        msg = Message(content=result, role=self.profile, cause_by=type(todo))
        self.rc.memory.add(msg)
        return msg

In [8]:
async def main():
    msg = "write a function that calculates the sum of a list"
    role = RunnableCoder()
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)

await main()

2024-05-17 02:59:01.924 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI
2024-05-17 02:59:01.928 | INFO     | __main__:main:4 - write a function that calculates the sum of a list
2024-05-17 02:59:01.933 | INFO     | __main__:_act:16 - Alice(RunnableCoder): 准备 SimpleWriteCode


 ```python
def sum_of_list(lst):
    return sum(lst)

# Test case 1
test_list1 = [1, 2, 3, 4, 5]
result1 = sum_of_list(test_list1)
print(result1)  # Expected output: 15

# Test case 2
test_list2 = [10, -5, 20, 0, 5]
result2 = sum_of_list(test_list2)
print(result2)  # Expected output: 20

```

2024-05-17 02:59:09.666 | INFO     | metagpt.utils.cost_manager:update_cost:48 - Total running cost: $0.001 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 52, completion_tokens: 143
2024-05-17 02:59:09.673 | INFO     | __main__:_act:16 - Alice(RunnableCoder): 准备 SimpleRunCode
2024-05-17 02:59:09.722 | INFO     | __main__:run:20 - code_result='15\n30\n'
2024-05-17 02:59:09.725 | INFO     | __main__:main:6 - RunnableCoder: 15
30



In [12]:
from typing import Dict, Union

class WriteDirectory(Action):
    """Action class for writing tutorial directories.

    Args:
        name: The name of the action.
        language: The language to output, default is "Chinese".
        
        用于编写教程目录的动作类。
        参数：
        name：动作的名称。
        language：输出的语言，默认为"Chinese"。
    """

    name: str = "WriteDirectory"
    language: str = "Chinese"

    async def run(self, topic: str, *args, **kwargs) -> Dict:
        """Execute the action to generate a tutorial directory according to the topic.

        Args:
            topic: The tutorial topic.

        Returns:
            the tutorial directory information, including {"title": "xxx", "directory": [{"dir 1": ["sub dir 1", "sub dir 2"]}]}.
        根据主题执行生成教程目录的操作。
            参数：
            topic：教程主题。
            返回：
            教程目录信息，包括{"title": "xxx", "directory": [{"dir 1": ["sub dir 1", "sub dir 2"]}]}.
        """
        COMMON_PROMPT = """
        You are now a seasoned technical professional in the field of the internet. 
        We need you to write a technical tutorial with the topic "{topic}".
        您现在是互联网领域的经验丰富的技术专业人员。
        我们需要您撰写一个关于"{topic}"的技术教程。
        """

        DIRECTORY_PROMPT = COMMON_PROMPT + """
        Please provide the specific table of contents for this tutorial, strictly following the following requirements:
        1. The output must be strictly in the specified language, {language}.
        2. Answer strictly in the dictionary format like {{"title": "xxx", "directory": [{{"dir 1": ["sub dir 1", "sub dir 2"]}}, {{"dir 2": ["sub dir 3", "sub dir 4"]}}]}}.
        3. The directory should be as specific and sufficient as possible, with a primary and secondary directory.The secondary directory is in the array.
        4. Do not have extra spaces or line breaks.
        5. Each directory title has practical significance.
        请按照以下要求提供本教程的具体目录：
        1. 输出必须严格符合指定语言，{language}。
        2. 回答必须严格按照字典格式，如{{"title": "xxx", "directory": [{{"dir 1": ["sub dir 1", "sub dir 2"]}}, {{"dir 2": ["sub dir 3", "sub dir 4"]}}]}}。
        3. 目录应尽可能具体和充分，包括一级和二级目录。二级目录在数组中。
        4. 不要有额外的空格或换行符。
        5. 每个目录标题都具有实际意义。
        """
        prompt = DIRECTORY_PROMPT.format(topic=topic, language=self.language)
        resp = await self._aask(prompt=prompt)
        return OutputParser.extract_struct(resp, dict)

In [13]:
def extract_struct(cls, text: str, data_type: Union[type(list), type(dict)]) -> Union[list, dict]:
    """Extracts and parses a specified type of structure (dictionary or list) from the given text.
    The text only contains a list or dictionary, which may have nested structures.

    Args:
        text: The text containing the structure (dictionary or list).
        data_type: The data type to extract, can be "list" or "dict".

    Returns:
        - If extraction and parsing are successful, it returns the corresponding data structure (list or dictionary).
        - If extraction fails or parsing encounters an error, it throw an exception.
    返回：
    - 如果提取和解析成功，它将返回相应的数据结构（列表或字典）。
    - 如果提取失败或解析遇到错误，则抛出异常。
    Examples:
        >>> text = 'xxx [1, 2, ["a", "b", [3, 4]], {"x": 5, "y": [6, 7]}] xxx'
        >>> result_list = OutputParser.extract_struct(text, "list")
        >>> print(result_list)
        >>> # Output: [1, 2, ["a", "b", [3, 4]], {"x": 5, "y": [6, 7]}]

        >>> text = 'xxx {"x": 1, "y": {"a": 2, "b": {"c": 3}}} xxx'
        >>> result_dict = OutputParser.extract_struct(text, "dict")
        >>> print(result_dict)
        >>> # Output: {"x": 1, "y": {"a": 2, "b": {"c": 3}}}
    """
    # Find the first "[" or "{" and the last "]" or "}"
    start_index = text.find("[" if data_type is list else "{")
    end_index = text.rfind("]" if data_type is list else "}")

    if start_index != -1 and end_index != -1:
        # Extract the structure part
        structure_text = text[start_index : end_index + 1]

        try:
            # Attempt to convert the text to a Python data type using ast.literal_eval
            result = ast.literal_eval(structure_text)

            # Ensure the result matches the specified data type
            if isinstance(result, list) or isinstance(result, dict):
                return result

            raise ValueError(f"The extracted structure is not a {data_type}.")

        except (ValueError, SyntaxError) as e:
            raise Exception(f"Error while extracting and parsing the {data_type}: {e}")
    else:
        logger.error(f"No {data_type} found in the text.")
        return [] if data_type is list else {}

In [14]:
class WriteContent(Action):
    """Action class for writing tutorial content.

    Args:
        name: The name of the action.
        directory: The content to write.
        language: The language to output, default is "Chinese".
    """

    name: str = "WriteContent"
    directory: dict = dict()
    language: str = "Chinese"

    async def run(self, topic: str, *args, **kwargs) -> str:
        """Execute the action to write document content according to the directory and topic.

        Args:
            topic: The tutorial topic.

        Returns:
            The written tutorial content.
        """
        COMMON_PROMPT = """
        You are now a seasoned technical professional in the field of the internet. 
        We need you to write a technical tutorial with the topic "{topic}".
        """
        CONTENT_PROMPT = COMMON_PROMPT + """
        Now I will give you the module directory titles for the topic. 
        Please output the detailed principle content of this title in detail. 
        If there are code examples, please provide them according to standard code specifications. 
        Without a code example, it is not necessary.

        The module directory titles for the topic is as follows:
        {directory}

        Strictly limit output according to the following requirements:
        1. Follow the Markdown syntax format for layout.
        2. If there are code examples, they must follow standard syntax specifications, have document annotations, and be displayed in code blocks.
        3. The output must be strictly in the specified language, {language}.
        4. Do not have redundant output, including concluding remarks.
        5. Strict requirement not to output the topic "{topic}".
        现在我将为您提供该主题的模块目录标题。
        请详细输出此标题的详细原理内容。
        如果有代码示例，请按照标准代码规范提供。
        没有代码示例则不需要提供。
        
        该主题的模块目录标题如下：
        {directory}
        
        严格按照以下要求限制输出：
        1. 遵循Markdown语法格式进行布局。
        2. 如果有代码示例，必须遵循标准语法规范，具备文档注释，并以代码块形式显示。
        3. 输出必须严格使用指定语言{language}。
        4. 不得有冗余输出，包括总结性陈述。
        5. 严禁输出主题"{topic}"。
        """
        prompt = CONTENT_PROMPT.format(
            topic=topic, language=self.language, directory=self.directory)
        return await self._aask(prompt=prompt)

In [15]:
class TutorialAssistant(Role):
    """Tutorial assistant, input one sentence to generate a tutorial document in markup format.

    Args:
        name: The name of the role.
        profile: The role profile description.
        goal: The goal of the role.
        constraints: Constraints or requirements for the role.
        language: The language in which the tutorial documents will be generated.
    """

    name: str = "Stitch"
    profile: str = "Tutorial Assistant"
    goal: str = "Generate tutorial documents"
    constraints: str = "Strictly follow Markdown's syntax, with neat and standardized layout"
    language: str = "Chinese"

    topic: str = ""
    main_title: str = ""
    total_content: str = ""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._init_actions([WriteDirectory(language=self.language)])
        self._set_react_mode(react_mode=RoleReactMode.REACT.value)

In [16]:
from datetime import datetime
from typing import Dict
import asyncio
from metagpt.actions.write_tutorial import WriteDirectory, WriteContent
from metagpt.const import TUTORIAL_PATH
from metagpt.logs import logger
from metagpt.roles.role import Role, RoleReactMode
from metagpt.schema import Message
from metagpt.utils.file import File

from typing import Dict

from metagpt.actions import Action
from metagpt.prompts.tutorial_assistant import DIRECTORY_PROMPT, CONTENT_PROMPT
from metagpt.utils.common import OutputParser

class WriteDirectory(Action):
    """Action class for writing tutorial directories.

    Args:
        name: The name of the action.
        language: The language to output, default is "Chinese".
    """

    name: str = "WriteDirectory"
    language: str = "Chinese"

    async def run(self, topic: str, *args, **kwargs) -> Dict:
        """Execute the action to generate a tutorial directory according to the topic.

        Args:
            topic: The tutorial topic.

        Returns:
            the tutorial directory information, including {"title": "xxx", "directory": [{"dir 1": ["sub dir 1", "sub dir 2"]}]}.
        """
        COMMON_PROMPT = """
        You are now a seasoned technical professional in the field of the internet. 
        We need you to write a technical tutorial with the topic "{topic}".
        """

        DIRECTORY_PROMPT = COMMON_PROMPT + """
        Please provide the specific table of contents for this tutorial, strictly following the following requirements:
        1. The output must be strictly in the specified language, {language}.
        2. Answer strictly in the dictionary format like {{"title": "xxx", "directory": [{{"dir 1": ["sub dir 1", "sub dir 2"]}}, {{"dir 2": ["sub dir 3", "sub dir 4"]}}]}}.
        3. The directory should be as specific and sufficient as possible, with a primary and secondary directory.The secondary directory is in the array.
        4. Do not have extra spaces or line breaks.
        5. Each directory title has practical significance.
        """
        prompt = DIRECTORY_PROMPT.format(topic=topic, language=self.language)
        resp = await self._aask(prompt=prompt)
        return OutputParser.extract_struct(resp, dict)

class WriteContent(Action):
    """Action class for writing tutorial content.

    Args:
        name: The name of the action.
        directory: The content to write.
        language: The language to output, default is "Chinese".
    """

    name: str = "WriteContent"
    directory: dict = dict()
    language: str = "Chinese"

    async def run(self, topic: str, *args, **kwargs) -> str:
        """Execute the action to write document content according to the directory and topic.

        Args:
            topic: The tutorial topic.

        Returns:
            The written tutorial content.
        """
        COMMON_PROMPT = """
        You are now a seasoned technical professional in the field of the internet. 
        We need you to write a technical tutorial with the topic "{topic}".
        """
        CONTENT_PROMPT = COMMON_PROMPT + """
        Now I will give you the module directory titles for the topic. 
        Please output the detailed principle content of this title in detail. 
        If there are code examples, please provide them according to standard code specifications. 
        Without a code example, it is not necessary.

        The module directory titles for the topic is as follows:
        {directory}

        Strictly limit output according to the following requirements:
        1. Follow the Markdown syntax format for layout.
        2. If there are code examples, they must follow standard syntax specifications, have document annotations, and be displayed in code blocks.
        3. The output must be strictly in the specified language, {language}.
        4. Do not have redundant output, including concluding remarks.
        5. Strict requirement not to output the topic "{topic}".
        """
        prompt = CONTENT_PROMPT.format(
            topic=topic, language=self.language, directory=self.directory)
        return await self._aask(prompt=prompt)

class TutorialAssistant(Role):
    """Tutorial assistant, input one sentence to generate a tutorial document in markup format.

    Args:
        name: The name of the role.
        profile: The role profile description.
        goal: The goal of the role.
        constraints: Constraints or requirements for the role.
        language: The language in which the tutorial documents will be generated.
    """

    name: str = "Stitch"
    profile: str = "Tutorial Assistant"
    goal: str = "Generate tutorial documents"
    constraints: str = "Strictly follow Markdown's syntax, with neat and standardized layout"
    language: str = "Chinese"

    topic: str = ""
    main_title: str = ""
    total_content: str = ""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._init_actions([WriteDirectory(language=self.language)])
        self._set_react_mode(react_mode=RoleReactMode.REACT.value)

    async def _think(self) -> None:
        """Determine the next action to be taken by the role."""
        logger.info(self.rc.state)
        logger.info(self,)
        if self.rc.todo is None:
            self._set_state(0)
            return

        if self.rc.state + 1 < len(self.states):
            self._set_state(self.rc.state + 1)
        else:
            self.rc.todo = None

    async def _handle_directory(self, titles: Dict) -> Message:
        """Handle the directories for the tutorial document.

        Args:
            titles: A dictionary containing the titles and directory structure,
                    such as {"title": "xxx", "directory": [{"dir 1": ["sub dir 1", "sub dir 2"]}]}

        Returns:
            A message containing information about the directory.
        """
        self.main_title = titles.get("title")
        directory = f"{self.main_title}\n"
        self.total_content += f"# {self.main_title}"
        actions = list()
        for first_dir in titles.get("directory"):
            actions.append(WriteContent(
                language=self.language, directory=first_dir))
            key = list(first_dir.keys())[0]
            directory += f"- {key}\n"
            for second_dir in first_dir[key]:
                directory += f"  - {second_dir}\n"
        self._init_actions(actions)
        self.rc.todo = None
        return Message(content=directory)

    async def _act(self) -> Message:
        """Perform an action as determined by the role.

        Returns:
            A message containing the result of the action.
        """
        todo = self.rc.todo
        if type(todo) is WriteDirectory:
            msg = self.rc.memory.get(k=1)[0]
            self.topic = msg.content
            resp = await todo.run(topic=self.topic)
            logger.info(resp)
            return await self._handle_directory(resp)
        resp = await todo.run(topic=self.topic)
        logger.info(resp)
        if self.total_content != "":
            self.total_content += "\n\n\n"
        self.total_content += resp
        return Message(content=resp, role=self.profile)

    async def _react(self) -> Message:
        """Execute the assistant's think and actions.

        Returns:
            A message containing the final result of the assistant's actions.
        """
        while True:
            await self._think()
            if self.rc.todo is None:
                break
            msg = await self._act()
        root_path = TUTORIAL_PATH / datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        await File.write(root_path, f"{self.main_title}.md", self.total_content.encode('utf-8'))
        return msg

async def main():
    msg = "Git 教程"
    role = TutorialAssistant()
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)

await main()

2024-05-17 03:46:42.033 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI
2024-05-17 03:46:42.035 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI
2024-05-17 03:46:42.036 | INFO     | __main__:main:202 - Git 教程
2024-05-17 03:46:42.037 | INFO     | __main__:_think:128 - -1
2024-05-17 03:46:42.038 | INFO     | __main__:_think:129 - name='Stitch' profile='Tutorial Assistant' goal='Generate tutorial documents' constraints="Strictly follow Markdown's syntax, with neat and standardized layout" desc='' is_human=False llm=<metagpt.provider.zhipuai_api.ZhiPuAILLM object at 0x7f02d6411ba0> role_id='' states=['0. WriteDirectory'] actions=[WriteDirectory] rc=RoleContext(env=None, msg_buffer=MessageQueue(), memory=Memory(storage=[user: Git 教程], index=defaultdict(<class 'list'>, {'metagpt.actions.add_requirement.UserRequirement': [user: Git 教程]}), ignore_id=False), state=-1, todo=None, watch={'metagpt.actions.add_

 {
    "title": "Git 教程",
    "directory": [
        {"dir 1": ["sub dir 1", "sub dir 2"]},
        {"dir 2": ["sub dir 3", "sub dir 4"]}
    ]
}

2024-05-17 03:46:45.635 | INFO     | metagpt.utils.cost_manager:update_cost:48 - Total running cost: $0.001 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 189, completion_tokens: 63
2024-05-17 03:46:45.638 | INFO     | __main__:_act:175 - {'title': 'Git 教程', 'directory': [{'dir 1': ['sub dir 1', 'sub dir 2']}, {'dir 2': ['sub dir 3', 'sub dir 4']}]}
2024-05-17 03:46:45.641 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI
2024-05-17 03:46:45.643 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI
2024-05-17 03:46:45.645 | INFO     | __main__:_think:128 - 0
2024-05-17 03:46:45.647 | INFO     | __main__:_think:129 - name='Stitch' profile='Tutorial Assistant' goal='Generate tutorial documents' constraints="Strictly follow Markdown's syntax, with neat and standardized layout" desc='' is_human=False llm=<metagpt.provider.zhipuai_api.ZhiPuAILLM object at 0x7f02d6411ba0> role_id='' states=['0. Wr

 ## 目录

### 1. 简介

Git 是一款强大的分布式版本控制系统，用于追踪文件更改和协调多人之间的工作。本教程将介绍 Git 的基本概念、操作和实用技巧。

### 2. 安装与配置 Git

#### 2.1 安装 Git

- Windows 用户可以访问 [Git 官网](https://git-scm.com/downloads) 下载适用于 Windows 的 Git 安装程序。
- macOS 用户可以使用内置的软件包管理器安装 Git，如 Homebrew。
- Linux 用户可以利用包管理器（如 apt、yum 或 pacman）安装 Git。

#### 2.2 配置 Git

- 打开命令行或终端，运行 `git config --global user.name "你的名字"` 和 `git config --global user.email "你的邮箱"` 进行基本配置。
- 设置 Git 参数，如压缩算法、钩子等，可以参考 [Git 配置文档](https://git-scm.com/docs/git-config)。

### 3. Git 仓库管理

#### 3.1 初始化仓库

在文件夹中运行 `git init` 命令初始化 Git 仓库。

#### 3.2 添加文件

使用 `git add` 命令将文件添加到暂存区。

#### 3.3 提交更改

运行 `git commit -m "提交信息"` 将暂存区的更改提交到本地仓库。

#### 3.4 查看仓库状态

运行 `git status` 命令查看当前仓库的状态，包括未添加的文件、已修改的文件等。

#### 3.5 删除文件

使用 `git rm` 命令删除文件，然后提交更改。

#### 3.6 分支管理

- 创建分支：运行 `git checkout -b 分支名`。
- 切换分支：运行 `git checkout 分支名`。
- 合并分支：运行 `git merge 分支名`。
- 删除分支：运行 `git branch -d 分支名`。

### 4. Git 远程仓库

#### 4.1 克隆远程仓库

运行 `git clone 远程仓库地址` 将远程仓库克隆到本地。

#### 4.2 推送和拉取仓库

- 推送：运行 `git 

2024-05-17 03:47:34.172 | INFO     | metagpt.utils.cost_manager:update_cost:48 - Total running cost: $0.002 | Max budget: $10.000 | Current cost: $0.001, prompt_tokens: 244, completion_tokens: 1019
2024-05-17 03:47:34.175 | INFO     | __main__:_act:178 -  ## 目录

### 1. 简介

Git 是一款强大的分布式版本控制系统，用于追踪文件更改和协调多人之间的工作。本教程将介绍 Git 的基本概念、操作和实用技巧。

### 2. 安装与配置 Git

#### 2.1 安装 Git

- Windows 用户可以访问 [Git 官网](https://git-scm.com/downloads) 下载适用于 Windows 的 Git 安装程序。
- macOS 用户可以使用内置的软件包管理器安装 Git，如 Homebrew。
- Linux 用户可以利用包管理器（如 apt、yum 或 pacman）安装 Git。

#### 2.2 配置 Git

- 打开命令行或终端，运行 `git config --global user.name "你的名字"` 和 `git config --global user.email "你的邮箱"` 进行基本配置。
- 设置 Git 参数，如压缩算法、钩子等，可以参考 [Git 配置文档](https://git-scm.com/docs/git-config)。

### 3. Git 仓库管理

#### 3.1 初始化仓库

在文件夹中运行 `git init` 命令初始化 Git 仓库。

#### 3.2 添加文件

使用 `git add` 命令将文件添加到暂存区。

#### 3.3 提交更改

运行 `git commit -m "提交信息"` 将暂存区的更改提交到本地仓库。

#### 3.4 查看仓库状态

运行 `git status` 命令查看当前仓库的状态，包括未添加的文件、已修改的文件等。

#### 3.5 删除文件

使用 `git rm

 # Git 教程

## 1. 简介

Git 是一款强大的分布式版本控制系统，用于追踪文件更改和协调多人之间的工作。本教程将介绍 Git 的基本概念、操作和实用技巧。

## 2. 安装与配置 Git

### 2.1 安装 Git

在 Windows 平台上，可以通过 [Git 官网](https://git-scm.com/downloads) 下载适用于 Windows 的 Git 安装程序。安装完成后，需配置 Git 用户名和电子邮件。

### 2.2 配置 Git

#### 2.2.1 设置用户名

打开命令行窗口，输入以下命令：

```
git config --global user.name "你的名字"
```

#### 2.2.2 设置电子邮件

输入以下命令：

```
git config --global user.email "你的电子邮件"
```

## 3. Git 仓库操作

### 3.1 初始化仓库

在文件夹中打开命令行窗口，输入以下命令：

```
git init
```

### 3.2 克隆仓库

克隆远程仓库至本地：

```
git clone 远程仓库地址
```

### 3.3 添加文件

将文件添加至暂存区：

```
git add 文件名
```

### 3.4 提交更改

将暂存区的更改提交至本地仓库：

```
git commit -m "提交信息"
```

### 3.5 查看仓库信息

查看当前分支：

```
git branch
```

查看提交历史：

```
git log
```

## 4. 分支管理

### 4.1 创建分支

创建新分支：

```
git checkout -b 新分支名
```

### 4.2 切换分支

切换至目标分支：

```
git checkout 目标分支名
```

### 4.3 合并分支

将当前分支合并至目标分支：

```
git merge 目标分支名
```

## 5. 远程仓库操作

### 5.1 添加远程仓库

```
git remote add 远程仓库名 远程仓库地址
```

### 5.2 推送本地仓库至远程仓库

```
git push -u 远程仓库名 本地分支名
```

### 

2024-05-17 03:48:23.239 | INFO     | metagpt.utils.cost_manager:update_cost:48 - Total running cost: $0.002 | Max budget: $10.000 | Current cost: $0.001, prompt_tokens: 244, completion_tokens: 1024
2024-05-17 03:48:23.242 | INFO     | __main__:_act:178 -  # Git 教程

## 1. 简介

Git 是一款强大的分布式版本控制系统，用于追踪文件更改和协调多人之间的工作。本教程将介绍 Git 的基本概念、操作和实用技巧。

## 2. 安装与配置 Git

### 2.1 安装 Git

在 Windows 平台上，可以通过 [Git 官网](https://git-scm.com/downloads) 下载适用于 Windows 的 Git 安装程序。安装完成后，需配置 Git 用户名和电子邮件。

### 2.2 配置 Git

#### 2.2.1 设置用户名

打开命令行窗口，输入以下命令：

```
git config --global user.name "你的名字"
```

#### 2.2.2 设置电子邮件

输入以下命令：

```
git config --global user.email "你的电子邮件"
```

## 3. Git 仓库操作

### 3.1 初始化仓库

在文件夹中打开命令行窗口，输入以下命令：

```
git init
```

### 3.2 克隆仓库

克隆远程仓库至本地：

```
git clone 远程仓库地址
```

### 3.3 添加文件

将文件添加至暂存区：

```
git add 文件名
```

### 3.4 提交更改

将暂存区的更改提交至本地仓库：

```
git commit -m "提交信息"
```

### 3.5 查看仓库信息

查看当前分支：

```
git branch
```

查看提交历史：

```
git log
```

## 4. 分支管理

### 4.1 创建分支

创建新分支：

```
gi